In [74]:
# install.packages("neuralnet")
# install.packages("mltools")
# install.packages("caret", dependencies = TRUE)

library(neuralnet)
library(mltools)
library(data.table)
library(caret)


In [75]:
iris2 <- scale(iris[, 1:4])
iris2 <- as.data.frame(iris2)

iris2$Species <- as.factor(iris$Species)

head(iris2)


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,-0.8976739,1.01560199,-1.335752,-1.311052,setosa
2,-1.1392005,-0.13153881,-1.335752,-1.311052,setosa
3,-1.3807271,0.32731751,-1.392399,-1.311052,setosa
4,-1.5014904,0.09788935,-1.279104,-1.311052,setosa
5,-1.0184372,1.24503015,-1.335752,-1.311052,setosa
6,-0.5353840,1.93331463,-1.165809,-1.048667,setosa


In [76]:
# Split the data into training and test sets
set.seed(1234)


In [77]:
particao <- createDataPartition(seq_len(dim(iris2)[1]), p = 0.7)
iris2_train <- iris2[particao$Resample1, ]
iris2_test <- iris2[-particao$Resample1, ]

dim(iris2_train)


[1] 106   5

In [78]:
iris2_train <- cbind(iris2_train[, 1:4], one_hot(as.data.table(iris2_train$Species)))

head(iris2_train)


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,V1_setosa,V1_versicolor,V1_virginica
,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
2,-1.139200,-0.13153881,-1.335752,-1.311052,1,0,0
3,-1.380727,0.32731751,-1.392399,-1.311052,1,0,0
4,-1.501490,0.09788935,-1.279104,-1.311052,1,0,0
5,-1.018437,1.24503015,-1.335752,-1.311052,1,0,0
6,-0.535384,1.93331463,-1.165809,-1.048667,1,0,0
7,-1.501490,0.78617383,-1.335752,-1.179859,1,0,0


In [79]:
modelo <- neuralnet(V1_setosa + V1_versicolor + V1_virginica ~ Sepal.Length + Sepal.Width + Petal.Length + Petal.Width, iris2_train, hidden = c(5, 4))

print(modelo)


$call
neuralnet(formula = V1_setosa + V1_versicolor + V1_virginica ~ 
    Sepal.Length + Sepal.Width + Petal.Length + Petal.Width, 
    data = iris2_train, hidden = c(5, 4))

$response
    V1_setosa V1_versicolor V1_virginica
2           1             0            0
3           1             0            0
4           1             0            0
5           1             0            0
6           1             0            0
7           1             0            0
8           1             0            0
9           1             0            0
11          1             0            0
14          1             0            0
15          1             0            0
16          1             0            0
18          1             0            0
20          1             0            0
21          1             0            0
22          1             0            0
23          1             0            0
24          1             0            0
25          1             0         

In [80]:
plot(modelo)


In [81]:
test <- compute(modelo, iris2_test[, 1:4])

head(test$net.result)


1,0.9999490,0.001184915,-3.122372e-06
10,0.9999490,0.001184888,-3.099547e-06
12,0.9999490,0.001184916,-3.123149e-06
13,0.9999497,0.001183223,-1.868730e-06
17,0.9999490,0.001184914,-3.122079e-06
19,0.9999490,0.001184914,-3.122419e-06


In [82]:
result <- as.data.frame(test$net.result)

head(result)


,V1,V2,V3
,<dbl>,<dbl>,<dbl>
1,0.9999490,0.001184915,-3.122372e-06
10,0.9999490,0.001184888,-3.099547e-06
12,0.9999490,0.001184916,-3.123149e-06
13,0.9999497,0.001183223,-1.868730e-06
17,0.9999490,0.001184914,-3.122079e-06
19,0.9999490,0.001184914,-3.122419e-06


In [83]:
names(result) <- c("setosa", "versicolor", "virginica")

head(result)


,setosa,versicolor,virginica
,<dbl>,<dbl>,<dbl>
1,0.9999490,0.001184915,-3.122372e-06
10,0.9999490,0.001184888,-3.099547e-06
12,0.9999490,0.001184916,-3.123149e-06
13,0.9999497,0.001183223,-1.868730e-06
17,0.9999490,0.001184914,-3.122079e-06
19,0.9999490,0.001184914,-3.122419e-06


In [84]:
result$class <- colnames(result)[max.col(result, "first")]

# result
head(result)


,setosa,versicolor,virginica,class
,<dbl>,<dbl>,<dbl>,<chr>
1,0.9999490,0.001184915,-3.122372e-06,setosa
10,0.9999490,0.001184888,-3.099547e-06,setosa
12,0.9999490,0.001184916,-3.123149e-06,setosa
13,0.9999497,0.001183223,-1.868730e-06,setosa
17,0.9999490,0.001184914,-3.122079e-06,setosa
19,0.9999490,0.001184914,-3.122419e-06,setosa


In [85]:
confusao = table(iris2_test$Species, result$class)

confusao

            
             setosa versicolor virginica
  setosa         17          0         0
  versicolor      0          8         2
  virginica       0          1        16

In [86]:
taxa_acerto = sum(diag(confusao)) / sum(confusao)

taxa_acerto

[1] 0.9318182